In [1]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import scipy as sp
import scipy.stats as st
from scipy.special import expit
import time
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from torch_geometric.nn import GCNConv, GraphConv, TAGConv
import torch.nn.functional as F
from torch_geometric.utils import from_scipy_sparse_matrix
import torch
import pandas as pd

from sklearn.linear_model import LogisticRegression

from google.colab import files

from ipywidgets import interact, IntSlider
from IPython import display

import torch_geometric.utils
from torch_geometric.datasets import Amazon, Planetoid, WikipediaNetwork, WebKB, Actor

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
def SGC(features, adjacency, order):
  # retrieving the number of nodes from the adjacency matrix
  num_nodes = adjacency.shape[0]

  # adding self-loops to the adjacency matrix by adding an identity matrix of the same size
  adjacency_augmented = adjacency + sp.sparse.eye(num_nodes)

  # calculating the augmented degree of each node (degree + 1 due to self-loop)
  degree_augmented = np.array(adjacency_augmented.sum(axis=1)).flatten()

  # normalizing the augmented adjacency matrix
  # multiplying the inverse square root of the degree matrix on both sides of the augmented adjacency matrix
  adjacency_normalized = (
      sp.sparse.spdiags(degree_augmented ** (-0.5), 0, num_nodes, num_nodes)
      @ adjacency_augmented
      @ sp.sparse.spdiags(degree_augmented ** (-0.5), 0, num_nodes, num_nodes)
  )

  # initializing the filtered_features variable with the input features
  filtered_features = features

  # propagating the features through the normalized adjacency matrix 'order' number of times
  for _ in range(1, order + 1):
    filtered_features = adjacency_normalized @ filtered_features

  # returning the final filtered features after propagating through the graph
  return filtered_features

In [4]:
def ASGC(features, adjacency, order, regularization, return_coeffs=False, verbose=False):
  # extracting the number of nodes and features from the input features matrix
  num_nodes, num_features = features.shape

  # computing the degree of each node in the adjacency matrix
  degree = np.array(adjacency.sum(axis=1)).flatten()

  # handling nodes with degree 0 (isolated nodes)
  if 0 in degree:
    if verbose:
        print("Added self-loops where degree was 0.")
    # finding the indices of the nodes with degree 0
    zero_degree_indices = np.argwhere(degree == 0).flatten()
    # creating a vector to store the degree corrections
    degree_fix = np.zeros(num_nodes)
    degree_fix[zero_degree_indices] = 1
    degree[zero_degree_indices] = 1.0
    # updating the adjacency matrix to add self-loops to the isolated nodes
    adjacency = adjacency + sp.sparse.spdiags(degree_fix, 0, num_nodes, num_nodes)

  # normalizing the adjacency matrix
  diag = sp.sparse.spdiags(degree ** (-0.5), 0, num_nodes, num_nodes)
  adjacency_normalized = diag @ adjacency @ diag

  # initializing an empty array to store the propagated features
  feature_propagation = np.empty((order + 1, num_nodes, num_features))
  feature_propagation[0, :, :] = features

  # propagating the features through the normalized adjacency matrix
  for i in range(1, order + 1):
    feature_propagation[i, :, :] = adjacency_normalized @ feature_propagation[i - 1, :, :]

  # initializing empty arrays for the coefficients and filtered features
  coefficients = np.empty((num_features, order + 1))
  filtered_features = np.empty((num_features, num_nodes))

  # transposing the feature_propagation matrix for easier manipulation
  feature_propagation = np.transpose(feature_propagation, (2, 1, 0))
  
  # creating the regularization vector
  regularization_vector = np.zeros(order + 1)
  regularization_vector[0] = np.sqrt(num_nodes * regularization)

  # iterating through each feature
  for feature_idx in range(num_features):
    # stacking the feature propagation and regularization vector together
    stacked_matrix = np.vstack((feature_propagation[feature_idx, :, :], regularization_vector[None, :]))

    # creating a target vector with the first column of feature propagation and zeros at the end
    target_vector = np.append(feature_propagation[feature_idx, :, 0], np.zeros(1))

    # solving the least squares problem to find the coefficients for the current feature
    coefficients[feature_idx, :], _, _, _ = np.linalg.lstsq(stacked_matrix, target_vector, rcond=None)

    # applying the coefficients to the feature propagation matrix to get the filtered feature values
    filtered_features[feature_idx, :] = feature_propagation[feature_idx, :, :] @ coefficients[feature_idx, :]

    # printing progress if verbose is set to True
    if verbose:
        print(f"Finished feature {feature_idx} of {num_features}.")

  # transposing the filtered_features matrix to match the input shape
  filtered_features = filtered_features.T

  # returning the filtered features and coefficients if requested, otherwise just the filtered features
  if return_coeffs:
    return filtered_features, coefficients
  else:
    return filtered_features

In [5]:
def stochasticblock(n, communities, edge_probs, return_exp=False):
  # initializing the expected adjacency matrix with zeros
  adj_exp = np.zeros((n, n))

  # determining the number of communities
  num_communities = len(communities)

  # iterating through the edge probabilities
  for (i, j), prob in np.ndenumerate(edge_probs):
      # updating the expected adjacency matrix using the edge probabilities
      adj_exp[np.ix_(communities[i], communities[j])] = prob

  # returning the expected adjacency matrix if requested
  if return_exp:
      return adj_exp
  else:
      # initializing the actual adjacency matrix with zeros
      adj = np.zeros((n, n), dtype=np.int8)

      # generating the actual adjacency matrix using the expected adjacency matrix
      adj[np.triu_indices(n)] = np.random.rand(n * (n + 1) // 2) < adj_exp[np.triu_indices(n)]

      # making the adjacency matrix symmetric
      adj = adj + adj.T

      # handling the diagonal elements (self-loops)
      adj[np.diag_indices(n)] = adj[np.diag_indices(n)] // 2

      # returning the actual adjacency matrix
      return adj

In [6]:
def make_sbm_feat(n,p,q, noise_sigma=1., matshow=False):
  # initializing the node labels
  labels = np.zeros(n)
  labels[n // 2:] = +1

  # generating the adjacency matrix using the stochastic block model function
  adj = stochasticblock(n, [np.arange(n // 2), np.arange(n // 2, n)], np.array([[p, q], [q, p]]))

  # visualizing the adjacency matrix if requested
  if matshow:
    matfig = plt.figure(figsize=(8, 3), dpi=300)
    plt.matshow(adj, fignum=matfig.number, aspect=1)

  # creating the true features based on the labels
  feats_true = 2. * labels - 1.

  # adding noise to the true features to generate the observed features
  feats = feats_true + np.random.normal(0, noise_sigma, size=n)

  # converting the adjacency matrix to a sparse CSR matrix and returning it with the true and observed features
  return sp.sparse.csr_matrix(adj), feats_true, feats

In [ ]:
n = 1000
sum_exp = 10 * 10 / n
p_q_logits = np.linspace(-2.5, 2.5, 3)

#setting up the figure with 1 row and 4 columns of subplots
fig, axs = plt.subplots(1, 4, figsize=(17 / 1.5, 5 / 1.5), dpi=100, tight_layout=True)
titles = ['Heterophilous', 'Neither', 'Homophilous']

#looping through each logit value to create corresponding plots
for i, p_q_logit in enumerate(p_q_logits):
    p_frac, q_frac = expit(p_q_logit), expit(-p_q_logit)
    
    #generating Stochastic Block Model (SBM) features with given probabilities
    adj, feats_true, feats = make_sbm_feat(n=n, p=p_frac * sum_exp, q=q_frac * sum_exp, noise_sigma=0.5)
    
    #setting title and displaying adjacency matrix as heatmap
    axs[i+1].set_title(titles[i])
    axs[i+1].matshow(np.array(adj.todense()), cmap='Blues', vmin=0, vmax=1)
    axs[i+1].set_xticks([0, int(n/2), n])
    axs[i+1].set_yticks([0, int(n/2), n])
    axs[i+1].xaxis.set_ticks_position("bottom")

#plotting features as scatter plot
axs[0].scatter(np.arange(int(n/2)), feats[:int(n/2)], s=1, color='indianred')
axs[0].scatter(np.arange(int(n/2), n), feats[int(n/2):n], s=1, color='steelblue')

#adding horizontal lines at y = -1 and y = 1
axs[0].plot([0, int(n/2)], [-1, -1], color='red')
axs[0].plot([int(n/2), n], [+1, +1], color='blue')

#setting grid, axis limits, and ticks for the scatter plot
axs[0].grid()
axs[0].set_ylim(-4, 4)
axs[0].set_xlim(0, n)
axs[0].set_xticks([0, int(n/2), n])

#setting labels for the scatter plot
axs[0].set_xlabel("Node")
axs[0].set_ylabel("Feature")
axs[0].set_aspect(abs(n - 0) / abs(4 - (-4)))
axs[0].set_title("Features distributions")

#displaying the plots
#plt.savefig("features.png")
#files.download("features.png") 
plt.show()


In [ ]:
#setting the number of nodes in the graph
num_nodes = 1000
#calculating the sum of probabilities p and q
pq_sum = 10 / num_nodes
#setting the lambda2 constant
lambda2 = -0.9
#calculating the individual probabilities p and q (inner and outer connections probabilities)
p = (pq_sum + lambda2 * pq_sum) / 2
q = (pq_sum - lambda2 * pq_sum) / 2
#generating the features using the given probabilities
adjacency_matrix, true_features, generated_features = make_sbm_feat(num_nodes, p, q, 1.0)
#computing the sum of the adjacency matrix along the given axis
degree = np.array(adjacency_matrix.sum(axis=1)).flatten()
#creating a copy of the degree array
degree_no_zero = degree.copy()
#replacing zero degrees with one
degree_no_zero[degree == 0] = 1
#defining minimum and maximum iterations for the precomputation (values of K, filterig steps)
min_iterations = 1
max_iterations = 10

precomputed_histograms = {}

#looping through the range of iterations
for iterations in range(min_iterations, max_iterations + 1):
    #applying Simple Graph Convolution (SGC) to the generated features
    features_SGC = SGC(generated_features, adjacency_matrix, iterations)
    #applying Adaptive Simple Graph Convolution (ASGC) to the generated features
    features_ASGC, coefficients = ASGC(generated_features.reshape(num_nodes, 1), adjacency_matrix, iterations, 1e10, True, False)
    features_ASGC = features_ASGC.flatten()
    
    #storing the generated features, SGC features, and ASGC features in a list
    data = [generated_features, features_SGC, features_ASGC]
    #adding the list of features to the precomputed_histograms dictionary
    precomputed_histograms[iterations] = data

#defining the plot_histograms function that takes 'iterations' as input
def plot_histograms(iterations):
    #creating a figure with 3 subplots
    fig, axes = plt.subplots(3, 1, figsize=(5, 6), dpi=80, tight_layout=True, sharex=True, sharey=True)

    #defining colors and titles for the histograms
    colors = {'raw': 'indianred', 'sgc': 'steelblue', 'asgc': 'darkorange'}
    titles = ['Raw', 'SGC', 'ASGC']
    #retrieving the precomputed data for the given number of iterations
    data = precomputed_histograms[iterations]

    #setting the opacity (alpha) and bin size for the histograms
    alpha = 0.9
    bins = np.linspace(-4, 4, 41)

    #looping through the subplots and plotting the histograms
    for idx, ax in enumerate(axes):
        #plotting the histograms for the first half of the dataset (red) and the second half (blue)
        ax.hist(data[idx][:int(num_nodes / 2)], alpha=alpha, bins=bins, color=colors['raw'])
        ax.hist(data[idx][int(num_nodes / 2):], alpha=alpha, bins=bins, color=colors['sgc'])
        #adding vertical dashed lines for red and blue means
        ax.axvline(-1, color='red', linestyle='--')
        ax.axvline(+1, color='blue', linestyle='--')
        
        #calculating and displaying mean and standard deviation for
        red_mean = np.mean(data[idx][:int(num_nodes / 2)])
        blue_mean = np.mean(data[idx][int(num_nodes / 2):])
        red_std = np.std(data[idx][:int(num_nodes / 2)])
        blue_std = np.std(data[idx][int(num_nodes / 2):])
        title = f"{titles[idx]} \n (R: μ={red_mean:.2f}, σ={red_std:.2f} | B: μ={blue_mean:.2f}, σ={blue_std:.2f})"
        ax.set_title(title)

        ax.set_xlim(-4, 4)
        ax.set_ylim(0, 150)
        for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                     ax.get_xticklabels() + ax.get_yticklabels()):
            item.set_fontsize(item.get_fontsize() * 1.2)

    #lt.show()

interact(plot_histograms, iterations=IntSlider(min=min_iterations, max=max_iterations, step=1, value=0, continuous_update=False))

In [ ]:
#setting the number of nodes in the graph
num_nodes = 1000
#calculating the sum of probabilities p and q
pq_sum = 10 / num_nodes
#setting the lambda2 constant
lambda2 = -0.9
#calculating the individual probabilities p and q (inner and outer connections probabilities)
p = (pq_sum + lambda2 * pq_sum) / 2
q = (pq_sum - lambda2 * pq_sum) / 2
#generating the features using the given probabilities
adjacency_matrix, true_features, generated_features = make_sbm_feat(num_nodes, p, q, 1.0)
#computing the sum of the adjacency matrix along the given axis
degree = np.array(adjacency_matrix.sum(axis=1)).flatten()
#creating a copy of the degree array
degree_no_zero = degree.copy()
#replacing zero degrees with one
degree_no_zero[degree == 0] = 1
#defining minimum and maximum iterations for the precomputation (values of K, filterig steps)
min_iterations = 1
max_iterations = 10
alpha = 0.9
bins = np.linspace(-4,4,41)

fig, axs = plt.subplots(3,4, figsize=(8,6), dpi=100, tight_layout=True, sharex=True, sharey=False)

feats_list_sgc = []
feats_list_asgc = []

for i, K in enumerate([1,2,4,8]):
    feats_list_sgc.append(SGC(generated_features, adjacency_matrix, K))
    features_ASGC, coefficients = ASGC(generated_features.reshape(num_nodes, 1), adjacency_matrix, K, 1e10, True, False)
    feats_list_asgc.append(features_ASGC.flatten())

    axs[0,i].hist(generated_features[:int(num_nodes/2)], alpha=alpha, bins=bins, color='indianred')
    axs[0,i].hist(generated_features[int(num_nodes/2):], alpha=alpha, bins=bins, color='steelblue')
    axs[0,i].axvline(-1, color='red', linestyle='--')
    axs[0,i].axvline(+1, color='blue', linestyle='--')
    axs[0,i].set_title(f"Raw, K={K}")
    axs[0,i].set_ylim(0,150)

    axs[1,i].hist(feats_list_sgc[i][:int(num_nodes/2)], alpha=alpha, bins=bins, color='indianred')
    axs[1,i].hist(feats_list_sgc[i][int(num_nodes/2):], alpha=alpha, bins=bins, color='steelblue')
    axs[1,i].axvline(-1, color='red', linestyle='--')
    axs[1,i].axvline(+1, color='blue', linestyle='--')
    axs[1,i].set_title(f"SGC, K={K}")
    axs[1,i].set_ylim(0,150)  # setting y limit to 200 for the second row

    axs[2,i].hist(feats_list_asgc[i][:int(num_nodes/2)], alpha=alpha, bins=bins, color='indianred')
    axs[2,i].hist(feats_list_asgc[i][int(num_nodes/2):], alpha=alpha, bins=bins, color='steelblue')
    axs[2,i].axvline(-1, color='red', linestyle='--')
    axs[2,i].axvline(+1, color='blue', linestyle='--')
    axs[2,i].set_title(f"ASGC, K={K}")
    axs[2,i].set_ylim(0,150)

for ax in axs.flat:
    ax.set_xlim(-4,4)

#plt.savefig("ks8.png")
#files.download("ks8.png") 
plt.show()

In [10]:
# Define the parameters
num_nodes = 1000
pq_sum = 10 / num_nodes
lambda2_vals = np.linspace(-1, 1, 51)
num_trials = 100
num_configs = len(lambda2_vals)
K = 2

# Define the arrays to store the results
sgc_mae = np.empty((num_configs, num_trials))
asgc_mae = np.empty((num_configs, num_trials))
raw_mae = np.empty((num_configs, num_trials))
sgc_sign = np.empty((num_configs, num_trials))
asgc_sign = np.empty((num_configs, num_trials))
raw_sign = np.empty((num_configs, num_trials))

# Generate the SBM graphs and calculate the SGC and ASGC scores for each graph
for i, lambda2 in enumerate(lambda2_vals):
    p = (pq_sum + lambda2 * pq_sum) / 2
    q = (pq_sum - lambda2 * pq_sum) / 2
    for trial in range(num_trials):
        adj_matrix, true_feats, noisy_feats = make_sbm_feat(
            n=num_nodes, p=p, q=q, noise_sigma=1.0)
        sgc_feats = SGC(noisy_feats, adj_matrix, K)
        asgc_feats = ASGC(noisy_feats.reshape(num_nodes, 1), adj_matrix, K, 1e10).flatten()
        sgc_mae[i, trial] = np.mean(np.abs(sgc_feats - true_feats))
        asgc_mae[i, trial] = np.mean(np.abs(asgc_feats - true_feats))
        raw_mae[i, trial] = np.mean(np.abs(noisy_feats - true_feats))
        sgc_sign[i, trial] = np.mean(np.sign(sgc_feats) == np.sign(true_feats))
        asgc_sign[i, trial] = np.mean(np.sign(asgc_feats) == np.sign(true_feats))
        raw_sign[i, trial] = np.mean(np.sign(noisy_feats) == np.sign(true_feats))


In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(5, 4), dpi=100, sharex=True, tight_layout=True)

#plotting the mean absolute errors for each method
axs[0].plot(lambda2_vals, np.ones(num_configs) * np.mean(raw_mae), label='Raw', linestyle='--', color='black', alpha=0.9)
sgc_line = axs[0].plot(lambda2_vals, np.mean(sgc_mae, axis=1), label='SGC', color='red', alpha=0.9)
asgc_line = axs[0].plot(lambda2_vals, np.mean(asgc_mae, axis=1), label='ASGC', color='blue', alpha=0.9)
axs[0].set_ylabel("Mean Absolute Error")
axs[0].grid()

#filling the area between the ASGC and SGC curves
sgc_y = np.mean(sgc_mae, axis=1)
asgc_y = np.mean(asgc_mae, axis=1)
axs[0].fill_between(lambda2_vals, sgc_y, asgc_y, where=asgc_y <= sgc_y, interpolate=True, color='blue', alpha=.25, label='Gain')
axs[0].fill_between(lambda2_vals, sgc_y, asgc_y, where=asgc_y > sgc_y, interpolate=True, color='red', alpha=.25, label='Loss')

#setting the x-axis labels for the second plot
axs[1].set_xlabel(r'$\frac{p-q}{p+q}$', labelpad=-5)
axs[1].set_xticks([-1.0, -0.5, 0, +0.5, +1])
axs[1].set_xticklabels(['-1.0\n(Heterophilous)', '-0.5', '0', '+0.5', '+1.0\n(Homophilous)'])
axs[1].grid()

#plotting the sign errors for each method
axs[1].plot(lambda2_vals, 1.-np.ones(num_configs)*raw_sign.mean(), label='Raw', linestyle='--', color='black', alpha=0.9)
axs[1].plot(lambda2_vals, 1. - np.mean(sgc_sign, axis=1), label='SGC', color='red', alpha=0.9)
axs[1].plot(lambda2_vals, 1. - np.mean(asgc_sign, axis=1), label='ASGC', color='blue', alpha=0.9)
axs[1].set_ylabel("Sign Error")
axs[1].grid()

#filling the area between the ASGC and SGC curves
sgc_y = 1. - np.mean(sgc_sign, axis=1)
asgc_y = 1. - np.mean(asgc_sign, axis=1)
axs[1].fill_between(lambda2_vals, sgc_y, asgc_y, where=asgc_y <= sgc_y, interpolate=True, color='blue', alpha=.25, label='Gain')
axs[1].fill_between(lambda2_vals, sgc_y, asgc_y, where=asgc_y > sgc_y, interpolate=True, color='red', alpha=.25, label='Loss')
axs[1].grid()

#adding the legend and adjust the plot layout
handles, labels = axs[1].get_legend_handles_labels()
lgd = fig.legend(handles, labels, bbox_to_anchor=[1.125, 0.65])
plt.tight_layout()
#plt.savefig("errors.png", bbox_inches='tight')
#files.download("errors.png") 
plt.show()

In [12]:
def load_dataset(dataset_name):
    # loading Amazon datasets
    if dataset_name in ['computers', 'photo']:
        data = Amazon('./', dataset_name).data
    # loading Planetoid datasets
    elif dataset_name in ['cora', 'citeseer', 'pubmed']:
        data = Planetoid('./', dataset_name).data
    # loading WikipediaNetwork datasets
    elif dataset_name in ['chameleon', 'squirrel']:
        data = WikipediaNetwork('./', dataset_name).data
    # loading Actor dataset
    elif dataset_name in ['film']:
        data = Actor('./', dataset_name).data
    # loading WebKB datasets
    elif dataset_name in ['cornell', 'texas']:
        data = WebKB('./', dataset_name).data
    else:
        raise ValueError(f"Unknown dataset: {dataset_name}")

    # converting edge_index to scipy sparse matrix
    adj = torch_geometric.utils.to_scipy_sparse_matrix(data.edge_index)
    # extracting features and labels
    feats = np.array(data.node_stores[0]['x'])
    labels = np.array(data.node_stores[0]['y'])
    # making sure adjacency matrix is symmetric
    adj = adj.maximum(adj.T)

    return adj, feats, labels


# defining names of homophilic datasets
homoph_dsets = ['cora', 'citeseer', 'pubmed', 'computers', 'photo']
# defining names of heterophilic datasets
heteroph_dsets = ['chameleon', 'squirrel', 'film', 'texas', 'cornell']
all_dsets = homoph_dsets + heteroph_dsets

# predownloading the datasets
for d in all_dsets:
    _, _, _ = load_dataset(d)

# defining a function to calculate the gap between the mean and upper bound of the 95% confidence interval
conf_int_95 = lambda a: st.t.interval(0.95, len(a) - 1, loc=np.mean(a), scale=st.sem(a))[1] - np.mean(a)

/usr/local/lib/python3.10/dist-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [13]:
# defining a function to load and process a dataset
def process_dataset(dset):
    # loading dataset
    adj, feats, labels = load_dataset(dset)
    # calculating the number of nodes
    n = adj.shape[0]
    # calculating the number of features and labels
    num_features, num_labels = feats.shape[1], len(set(labels))
    # printing out the dataset details
    print(
        f"Dataset '{dset}' loaded. There are {n} nodes, {(adj[np.triu_indices(n)] > 0).sum()} edges, {num_features} features, and {num_labels} labels.")
    # setting proportions for test and validation sets depending on the type of dataset
    test_prop, valid_prop = (0.95, 0.5) if dset in homoph_dsets else (0.2, 0.25)
    # returning processed data
    return adj, feats, labels, n, test_prop, valid_prop


# defining a function to calculate accuracy with given features
def get_acc_with_feats(filtered_feats, random_state, valid, nontest_idcs, test_idcs, labels):
    # splitting data into training and validation sets if 'valid' is True
    if valid:
        train_idcs, valid_idcs = train_test_split(nontest_idcs, test_size=valid_prop, random_state=random_state)
        # fitting logistic regression model and calculating accuracy for validation set
        clf = LogisticRegression(max_iter=logreg_maxiter).fit(filtered_feats[train_idcs, :], labels[train_idcs])
        return (labels[valid_idcs] == clf.predict(filtered_feats[valid_idcs, :])).mean()
    else:
        # fitting logistic regression model and calculating accuracy for test set if 'valid' is False
        clf = LogisticRegression(max_iter=logreg_maxiter).fit(filtered_feats[nontest_idcs, :], labels[nontest_idcs])
        return (labels[test_idcs] == clf.predict(filtered_feats[test_idcs, :])).mean()


# using raw features
def calculate_raw_test_acc(dset, feats, random_states, n, test_prop, labels):
    # initializing an empty array to store raw test accuracies
    raw_test_accs[dset] = np.empty((num_trials))
    raw_tt_time[dset] = np.empty((num_trials))
    # iterating over each random state
    for i, random_state in enumerate(random_states):
        # splitting indices into non-test and test sets
        t = time.time()
        nontest_idcs, test_idcs = train_test_split(np.arange(n), test_size=test_prop, random_state=random_state)
        # computing and storing the test accuracy for each trial using raw features
        raw_test_accs[dset][i] = get_acc_with_feats(feats, random_state, valid=False, nontest_idcs=nontest_idcs,
                                                    test_idcs=test_idcs, labels=labels)
        raw_tt_time[dset][i] = time.time() - t
    # printing the mean test accuracy and the 95% confidence interval
    print(
        f"\tTest accuracy with raw is {100 * raw_test_accs[dset].mean():.2f} +/- {100 * conf_int_95(raw_test_accs[dset]):.2f}")


# defining a function to calculate test accuracy using simple graph convolution (SGC)
def calculate_sgc_test_acc(dset, feats, adj, random_states, n, test_prop, valid_prop, labels):
    # initializing empty arrays to store validation accuracies, best K indices and test accuracies for SGC
    sgc_valid_accs[dset] = np.empty((num_trials, len(K_vals)))
    sgc_best_K_idcs[dset] = np.empty((num_trials), dtype=int)
    sgc_test_accs[dset] = np.empty((num_trials))
    sgc_tt_time[dset] = np.empty((num_trials))
    # iterating over each random state
    for i, random_state in enumerate(random_states):
        # splitting indices into non-test and test sets
        nontest_idcs, test_idcs = train_test_split(np.arange(n), test_size=test_prop, random_state=random_state)
        # iterating over each possible K value
        for K_num, K_val in enumerate(K_vals):
            # applying SGC to the features
            feats_filtered_sgc = SGC(feats, adj, K_val)
            # computing and storing the validation accuracy for each K
            sgc_valid_accs[dset][i, K_num] = get_acc_with_feats(feats_filtered_sgc, random_state, valid=True,
                                                                nontest_idcs=nontest_idcs, test_idcs=test_idcs,
                                                                labels=labels)
            # finding the K that gives the highest validation accuracy
        sgc_best_K_idcs[dset][i] = sgc_valid_accs[dset][i].argmax()
        t = time.time()
        # applying SGC to the features with the best K
        feats_filtered_sgc = SGC(feats, adj, K_vals[sgc_best_K_idcs[dset][i]])
        # computing and storing the test accuracy using the best K
        sgc_test_accs[dset][i] = get_acc_with_feats(feats_filtered_sgc, random_state, valid=False,
                                                    nontest_idcs=nontest_idcs, test_idcs=test_idcs, labels=labels)
        sgc_tt_time[dset][i] = time.time() - t

    # printing the mean test accuracy and the 95% confidence interval
    print(
        f"\tTest accuracy with SGC is {100 * sgc_test_accs[dset].mean():.2f} +/- {100 * conf_int_95(sgc_test_accs[dset]):.2f}")


def calculate_asgc_test_acc(dset, feats, adj, random_states, n, test_prop, valid_prop, labels):
    # initializing empty arrays to store validation accuracies, best K indices, best Rp indices, and test accuracies for ASGC
    asgc_valid_accs[dset] = np.empty((num_trials, len(K_vals), len(Rp_vals)))
    asgc_best_K_idcs[dset] = np.empty((num_trials), dtype=int)
    asgc_best_Rp_idcs[dset] = np.empty((num_trials), dtype=int)
    asgc_test_accs[dset] = np.empty((num_trials))
    asgc_tt_time[dset] = np.empty((num_trials))
    # iterating over each random state
    for i, random_state in enumerate(random_states):
        # splitting indices into non-test and test sets
        nontest_idcs, test_idcs = train_test_split(np.arange(n), test_size=test_prop, random_state=random_state)
        # iterating over each possible K value and regularization parameter Rp
        for K_num, K_val in enumerate(K_vals):
            for Rp_num, Rp_val in enumerate(Rp_vals):
                # applying ASGC to the features
                feats_filtered_asgc = ASGC(feats, adj, K_val, Rp_val)
                # computing and storing the validation accuracy for each K and Rp
                asgc_valid_accs[dset][i, K_num, Rp_num] = get_acc_with_feats(feats_filtered_asgc, random_state,
                                                                             valid=True, nontest_idcs=nontest_idcs,
                                                                             test_idcs=test_idcs, labels=labels)
        # finding the K and Rp that give the highest validation accuracy
        asgc_best_K_idcs[dset][i], asgc_best_Rp_idcs[dset][i] = np.unravel_index(asgc_valid_accs[dset][i].argmax(),
                                                                                 (len(K_vals), len(Rp_vals)))
        t = time.time()
        # applying ASGC to the features with the best K and Rp
        feats_filtered_asgc = ASGC(feats, adj, K_vals[asgc_best_K_idcs[dset][i]], Rp_vals[asgc_best_Rp_idcs[dset][i]])
        # computing and storing the test accuracy using the best K and Rp
        asgc_test_accs[dset][i] = get_acc_with_feats(feats_filtered_asgc, random_state, valid=False,
                                                     nontest_idcs=nontest_idcs, test_idcs=test_idcs, labels=labels)
        asgc_tt_time[dset][i] = time.time() - t

    # printing the mean test accuracy and the 95% confidence interval
    print(
        f"\tTest accuracy with ASGC is {100 * asgc_test_accs[dset].mean():.2f} +/- {100 * conf_int_95(asgc_test_accs[dset]):.2f}")


In [14]:
def create_mlp_model(input_dim, output_dim):
    # creating the MLP model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(4000, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(2000, activation='relu'),
        tf.keras.layers.Dense(output_dim, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


def calculate_mlp_test_acc(dset, feats, random_states, n, test_prop, valid_prop, labels):
    # Initializing an empty array to store MLP test accuracies
    mlp_test_accs[dset] = np.empty((len(random_states)))
    mlp_tt_time[dset] = np.empty((num_trials))
    # Transforming labels into one-hot vectors
    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
    # Iterating over each random state
    for i, random_state in enumerate(random_states):
        t = time.time()
        # Splitting indices into non-test and test sets
        nontest_idcs, test_idcs = train_test_split(np.arange(n), test_size=test_prop, random_state=random_state)
        # Creating the MLP model
        model = create_mlp_model(feats.shape[1], labels.shape[1])
        # Training the model with early stopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        model.fit(feats[nontest_idcs, :], labels[nontest_idcs], callbacks=[early_stopping], validation_split=valid_prop,
                  epochs=50, batch_size=32, verbose=0)
        # Evaluating the model and storing the test accuracy
        _, accuracy = model.evaluate(feats[test_idcs, :], labels[test_idcs], verbose=0)
        mlp_test_accs[dset][i] = accuracy
        mlp_tt_time[dset][i] = time.time() - t
    # Printing the mean test accuracy
    print(
        f"\tTest accuracy with MLP is {100 * mlp_test_accs[dset].mean():.2f} +/- {100 * conf_int_95(mlp_test_accs[dset]):.2f}")



In [15]:
class Net(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(Net, self).__init__()
        self.conv1 = GCNConv(num_features, 2000)
        # self.conv1 = TAGConv(num_features, num_features*2, K=2)
        self.conv2 = GCNConv(2000, num_classes)
        # self.conv2 = TAGConv(num_features*2, num_classes, K=2)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)


def train_and_test(dataset_name, seed, adj, feats, labels):
    torch.manual_seed(seed)

    # according to the paper
    if dataset_name in homoph_dsets:
        test_prop = 0.95
        valid_prop = 0.025
    else:
        test_prop = 0.2
        valid_prop = 0.2

    edge_index = from_scipy_sparse_matrix(adj)[0]
    x = torch.from_numpy(feats).float()
    y = torch.from_numpy(labels).long()

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net(x.size(1), len(np.unique(y))).to(device)
    x = x.to(device)
    edge_index = edge_index.to(device)
    y = y.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

    # calculate the indices for train, validation and test data
    train_index = torch.arange(0, int((1 - test_prop - valid_prop) * y.size(0)), dtype=torch.long).to(device)
    val_index = torch.arange(int((1 - test_prop - valid_prop) * y.size(0)), int((1 - test_prop) * y.size(0)),
                             dtype=torch.long).to(device)
    test_index = torch.arange(int((1 - test_prop) * y.size(0)), y.size(0), dtype=torch.long).to(device)

    highest_val = 0
    patience = 0

    for epoch in range(50):
        model.train()
        optimizer.zero_grad()
        out = model(x, edge_index)
        loss = F.nll_loss(out[train_index], y[train_index])
        loss.backward()
        optimizer.step()

        # Validation
        model.eval()
        _, pred = model(x, edge_index).max(dim=1)
        correct = float(pred[val_index].eq(y[val_index]).sum().item())
        val_acc = correct / val_index.size(0)

        if highest_val < val_acc:
            highest_val = val_acc
            patience = 0
        else:
            patience += 1

        if patience > 9:
            break

    model.eval()
    _, pred = model(x, edge_index).max(dim=1)
    correct = float(pred[test_index].eq(y[test_index]).sum().item())
    acc = correct / test_index.size(0)
    return acc


def calculate_gnn_test_acc(dset, random_states, adj, feats, labels):
    gcn_test_accs[dset] = np.empty((len(random_states)))
    gcn_tt_time[dset] = np.empty((num_trials))
    # Iterating over each random state
    for i, random_state in enumerate(random_states):
        t = time.time()
        gcn_test_accs[dset][i] = train_and_test(dset, random_state, adj, feats, labels)
        gcn_tt_time[dset][i] = time.time() - t

    # Printing the mean test accuracy
    print(f"\tTest accuracy with GCN is {100 * gcn_test_accs[dset].mean():.2f} +/- {100 * conf_int_95(gcn_test_accs[dset]):.2f}")


In [ ]:
# initializing dictionaries to store results of validations and tests
sgc_valid_accs, asgc_valid_accs = {}, {}
sgc_best_K_idcs, asgc_best_K_idcs, asgc_best_Rp_idcs = {}, {}, {}
raw_test_accs, sgc_test_accs, asgc_test_accs, mlp_test_accs, gcn_test_accs = {}, {}, {}, {}, {}
raw_tt_time, sgc_tt_time, asgc_tt_time, mlp_tt_time, gcn_tt_time = {}, {}, {}, {}, {}

# setting the maximum number of iterations for logistic regression
logreg_maxiter = 1000
num_trials = 10
# defining possible values for number of hops (K)
K_vals = np.array([1, 2, 3, 4, 6, 8])
# defining possible values for the zero coefficient regularization (Rp)
Rp_vals = np.geomspace(1e-6, 1e6, num=13)
# setting random states for the reproducibility of the experiment
random_states = 42 * np.arange(num_trials)

# Main loop for processing datasets
for dset in all_dsets:
    adj, feats, labels, n, test_prop, valid_prop = process_dataset(dset)

    calculate_raw_test_acc(dset, feats, random_states, n, test_prop, labels)
    calculate_sgc_test_acc(dset, feats, adj, random_states, n, test_prop, valid_prop, labels)
    calculate_asgc_test_acc(dset, feats, adj, random_states, n, test_prop, valid_prop, labels)
    calculate_mlp_test_acc(dset, feats, random_states, n, test_prop, valid_prop, labels)
    calculate_gnn_test_acc(dset, random_states, adj, feats, labels)

    print()

In [ ]:

df_raw = pd.DataFrame(raw_test_accs)
df_sgc = pd.DataFrame(sgc_test_accs)
df_asgc = pd.DataFrame(asgc_test_accs)
df_mlp = pd.DataFrame(mlp_test_accs)
df_gcn = pd.DataFrame(gcn_test_accs)
# Reset index
df_raw.reset_index(inplace=True)
df_sgc.reset_index(inplace=True)
df_asgc.reset_index(inplace=True)
df_mlp.reset_index(inplace=True)
df_gcn.reset_index(inplace=True)

#melting dataframes and add method column
df_raw = df_raw.melt(id_vars=['index'], var_name='dataset', value_name='Accuracy')
df_raw['method'] = 'raw'
df_sgc = df_sgc.melt(id_vars=['index'], var_name='dataset', value_name='Accuracy')
df_sgc['method'] = 'sgc'
df_asgc = df_asgc.melt(id_vars=['index'], var_name='dataset', value_name='Accuracy')
df_asgc['method'] = 'asgc'
df_mlp = df_mlp.melt(id_vars=['index'], var_name='dataset', value_name='Accuracy')
df_mlp['method'] = 'mlp'
df_gcn = df_gcn.melt(id_vars=['index'], var_name='dataset', value_name='Accuracy')
df_gcn['method'] = 'gcn'

#combining all dataframes
df_final = pd.concat([df_raw, df_sgc, df_asgc, df_mlp, df_gcn], ignore_index=True)
df_final.drop('index', axis=1, inplace=True)

#getting unique datasets
datasets = df_final['dataset'].unique()
df_final['method'] = df_final['method'].str.upper()

#creating a 5x2 grid of subplots
fig, axs = plt.subplots(5, 2, figsize=(10, 20), sharey=True)

# defining color palettes for the two different types of graphs
palette1 = sns.color_palette("Blues")
palette2 = sns.color_palette("Greens")

#creating the barplot for the accuracy
for i, dataset in enumerate(datasets):

    ax = axs[i % 5, i // 5]
    data = df_final[df_final['dataset'] == dataset]
    palette = palette1 if i < 5 else palette2
    sns.barplot(x='method', y='Accuracy', data=data, ax=ax, palette=palette)
    ax.set_title(dataset)

plt.tight_layout()
plt.savefig("./acc_barplot.png")
#files.download("./acc_barplot.png") 
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(7, 6), sharex=True)

#defining method groups
methods = df_final['method'].unique()
df_final['type'] = df_final['dataset'].apply(lambda x: 'Homophilous' if x in homoph_dsets else 'Heterophilous')

#creating the boxplots
sns.boxplot(x='method', y='Accuracy', data=df_final[df_final['type'] == 'Homophilous'], ax=axs[0], palette='Blues')
sns.boxplot(x='method', y='Accuracy', data=df_final[df_final['type'] == 'Heterophilous'], ax=axs[1], palette='Greens')

for ax in axs:
    ax.set_ylabel('Accuracy')
axs[0].set_title('Homophilous')
axs[1].set_title('Heterophilous')

plt.xticks(range(len(methods)), methods)
plt.tight_layout()
plt.savefig("./acc_boxplot.png")
#files.download("./acc_boxplot.png") 
plt.show()


In [ ]:
#we then do the same process for the time dicts
df_raw_time = pd.DataFrame(raw_tt_time)
df_sgc_time = pd.DataFrame(sgc_tt_time)
df_asgc_time = pd.DataFrame(asgc_tt_time)
df_mlp_time = pd.DataFrame(mlp_tt_time)
df_gcn_time = pd.DataFrame(gcn_tt_time)

df_raw_time.reset_index(inplace=True)
df_sgc_time.reset_index(inplace=True)
df_asgc_time.reset_index(inplace=True)
df_mlp_time.reset_index(inplace=True)
df_gcn_time.reset_index(inplace=True)

df_raw_time = df_raw_time.melt(id_vars=['index'], var_name='dataset', value_name='Time')
df_raw_time['method'] = 'raw'
df_sgc_time = df_sgc_time.melt(id_vars=['index'], var_name='dataset', value_name='Time')
df_sgc_time['method'] = 'sgc'
df_asgc_time = df_asgc_time.melt(id_vars=['index'], var_name='dataset', value_name='Time')
df_asgc_time['method'] = 'asgc'
df_mlp_time = df_mlp_time.melt(id_vars=['index'], var_name='dataset', value_name='Time')
df_mlp_time['method'] = 'mlp'
df_gcn_time = df_gcn_time.melt(id_vars=['index'], var_name='dataset', value_name='Time')
df_gcn_time['method'] = 'gcn'

df_final_time = pd.concat([df_raw_time, df_sgc_time, df_asgc_time, df_mlp_time, df_gcn_time], ignore_index=True)
df_final_time.drop('index', axis=1, inplace=True)

df_final_time['method'] = df_final_time['method'].str.upper()

fig, axs = plt.subplots(5, 2, figsize=(10, 20), sharey=False)

palette1 = sns.color_palette("Oranges")
palette2 = sns.color_palette("Purples")

for i, dataset in enumerate(datasets):

    ax = axs[i % 5, i // 5]
    data = df_final_time[df_final_time['dataset'] == dataset]
    palette = palette1 if i < 5 else palette2
    sns.barplot(x='method', y='Time', data=data, ax=ax, palette=palette)
    ax.set_title(dataset)

plt.tight_layout()
plt.savefig("./time_barplot.png")
#files.download("./time_barplot.png") 
plt.show()


In [ ]:
#we then do the same process for the time dicts
fig, axs = plt.subplots(2, 1, figsize=(7, 6), sharex=True)

methods = df_final['method'].unique()
df_final_time['type'] = df_final_time['dataset'].apply(
    lambda x: 'Homophilous' if x in homoph_dsets else 'Heterophilous')


sns.boxplot(x='method', y='Time', data=df_final_time[df_final_time['type'] == 'Homophilous'], ax=axs[0],
            palette='Oranges')
sns.boxplot(x='method', y='Time', data=df_final_time[df_final_time['type'] == 'Heterophilous'], ax=axs[1],
            palette='Purples')

for ax in axs:
    ax.set_ylabel('Time')
axs[0].set_title('Homophilous')
axs[1].set_title('Heterophilous')

plt.xticks(range(len(methods)), methods)
plt.tight_layout()
plt.savefig("./time_boxplot.png")
#files.download("./time_boxplot.png") 
plt.show()

In [ ]:
df_final.to_csv('./df_final_acc.csv')
df_final_time.to_csv('./df_final_time.csv')

mean_values = df_final.groupby('method')['Accuracy'].mean()
print(mean_values)
std_values = df_final.groupby('method')['Accuracy'].std()
print(std_values)

print('-----------------------')

mean_values = df_final_time.groupby('method')['Time'].mean()
print(mean_values)

std_values = df_final_time.groupby('method')['Time'].std()
print(std_values)